# Exercise 2: Transfer Learning

Attribution: Kolhatkar, Varada (2024) DSCI572 

## Imports
<hr>

<br><br>

In [ ]:
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
from torchvision import datasets, models, transforms, utils
import glob
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import pandas as pd
import random
import torch
import torchvision
%matplotlib inline

plt.rcParams.update({'axes.grid': False})

## Getting Started with Kaggle Kernels
<hr>

We are going to run this notebook on the cloud using [Kaggle](https://www.kaggle.com). Kaggle offers 30 hours of free GPU usage per week which should be much more than enough for this lab. To get started, follow these steps:

1. Go to https://www.kaggle.com/kernels

2. Make an account if you don't have one, and verify your phone number (to get access to GPUs)
3. Select `+ New Notebook`
4. Go to `File -> Import Notebook`
5. Upload this notebook
6. On the right-hand side of your Kaggle notebook, make sure:
  
  - `Internet` is enabled.
  
  - In the `Accelerator` dropdown, choose `GPU` when you're ready to use it (you can turn it on/off as you need it).
    
7. Run the follow cells for preparation the model, labels and functions.

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [ ]:
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def classify_image(img, topn = 4):
    clf = models.vgg16(weights='VGG16_Weights.DEFAULT') # initialize the classifier with VGG16 weights
    preprocess = transforms.Compose([
                 transforms.Resize(299),
                 transforms.CenterCrop(299),
                 transforms.ToTensor(),
                 transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225]),])

    with open("imagenet_classes.txt", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    img_t = preprocess(img)
    batch_t = torch.unsqueeze(img_t, 0)
    clf.eval()
    output = clf(batch_t)
    _, indices = torch.sort(output, descending=True)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    d = {'Class': [classes[idx] for idx in indices[0][:topn]], 
         'Probability score': [np.round(probabilities[0, idx].item(),3) for idx in indices[0][:topn]]}
    df = pd.DataFrame(d, columns = ['Class','Probability score'])
    return df


# Attribution: [Code from PyTorch docs](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html?highlight=transfer%20learning)
IMAGE_SIZE = 200
BATCH_SIZE = 64

def read_data(data_dir, subdir):
    """
    Reads image data from the specified directory and applies transformations.
    """
    data_transforms = {
        "train": transforms.Compose(
            [
                transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),     
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),            
            ]
        ),
        "valid": transforms.Compose(
            [
                transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),                        
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),                        
            ]
        ),
    }

    image_datasets = {
        x: datasets.ImageFolder(os.path.join(data_dir, subdir[x]), data_transforms[x])
        for x in ["train", "valid"]
    }
    
    dataloaders = {}
    
    dataloaders["train"] = torch.utils.data.DataLoader(
            image_datasets["train"], batch_size=BATCH_SIZE, shuffle=True
        )
    
    dataloaders["valid"] = torch.utils.data.DataLoader(
            image_datasets["valid"], batch_size=BATCH_SIZE, shuffle=True
        )
    
    return image_datasets, dataloaders

def get_features(model, data_loader, seed=None):
    """Extract output of squeezenet model"""
    if seed:
        torch.manual_seed(seed)
    model.to(device)
    with torch.no_grad():  # turn off computational graph stuff
        Z_init = torch.empty((0, 1024)).to(device)  # Initialize empty tensors
        y_init = torch.empty((0)).to(device)
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            Z_init = torch.cat((Z_init, model(X)), dim=0)
            y_init = torch.cat((y_init, y))
    return Z_init.cpu().detach(), y_init.cpu().detach()

def show_predictions(pipe, Z_valid, y_valid, dataloader, class_names, num_images=20, seed=None):
    """Display images from the validation set and their predicted labels."""
    if seed:
        torch.manual_seed(seed)
    images_so_far = 0
    fig = plt.figure(figsize=(15, 25))  # Adjust the figure size for better visualization

    # Convert the features and labels to numpy arrays
    Z_valid = Z_valid.numpy()
    y_valid = y_valid.numpy()

    # Make predictions using the trained logistic regression model
    preds = pipe.predict(Z_valid)

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.cpu()
            for j in range(inputs.size()[0]):
                if images_so_far >= num_images:
                    return
                # print(f"Dataloader Labels: {labels[j]}: {class_names['valid'][labels[j]]}")
                ax = plt.subplot(num_images // 5, 5, images_so_far + 1)  # 5 images per row
                ax.axis('off')
                ax.set_title(f"Predicted: {class_names['train'][int(preds[images_so_far])]}"
                             f"\nTrue: {class_names['valid'][int(y_valid[images_so_far])]}")
                inp = inputs.data[j].numpy().transpose((1, 2, 0))
                mean = np.array([0.5, 0.5, 0.5])
                std = np.array([0.5, 0.5, 0.5])
                inp = std * inp + mean
                inp = np.clip(inp, 0, 1)
                ax.imshow(inp)
                #imshow(inputs.data[j])
                images_so_far += 1

def show_image_label_prob(image_path, true_label, num_images):
    """
    Displays a specified number of images from a given directory and prints their classification labels and probabilities.
    """
    images = glob.glob(image_path)
    selected_images = random.sample(images, num_images)
    plt.figure(figsize=(5, 5));
    for image in selected_images:
        img = Image.open(image)
        img.load()
        plt.imshow(img)
        plt.title(f'Actual Label: {true_label}')
        plt.show()
        df = classify_image(img)    
        print(df.to_string(index=False))
        print("--------------------------------------------------------------")

Once you've done all your work on Kaggle, you can download the notebook from Kaggle. That way any work you did on Kaggle won't be lost. 

## Exercise 1: Transfer Learning
<hr>

In this exercise you're going to practice transfer learning. We're going to develop a model that can detect different fruits in this Kaggle [dataset](https://www.kaggle.com/datasets/karimabdulnabi/fruit-classification10-class):

In order to use this dataset 

1. Click `+ Add data` at the top right of the notebook.

2. Choose `Datasets`. In the search bar, type 'fruit-classification10-class'. You will see several datasets listed. Find and add the top dataset that has a size of 31 MB.

### Out-of-the-box Classification

BBefore starting the exercise, you will harness the power of a Convolutional Neural Network (CNN) model to classify images of cats and dogs using a common [dataset](https://www.kaggle.com/datasets/tongpython/cat-and-dog).

However, training a CNN model from scratch can be computationally intensive. Therefore, you will use a pre-trained model (`VGG` for this case), to identify the animals efficiently!

In order to use the dataset.

1. Click `+ Add data` at the top right of the notebook.

2. Choose `Datasets`. In the search bar, type 'cat-and-dog'. You will see several datasets listed. Find and add the top dataset that has a size of 228 MB.

By running the following cell, you will obtain the image, the model's predictions (`Class`), and the associated probabilities.

In [ ]:
# Set up data
IMAGE_DIR = {
    "Cat": "/kaggle/input/cat-and-dog/test_set/test_set/cats/*.*", 
    "Dog": "/kaggle/input/cat-and-dog/test_set/test_set/dogs/*.*"
}

# Display image and prediction labels with probability
for true_label, image_path in IMAGE_DIR.items():
    show_image_label_prob(image_path, true_label, num_images=4)

**Food for Thought**: 

- How effectively can the model distinguish between cats and dogs?
- Do you observe any particular characteristics about the labels?

_Type your answer here, replacing this text._

Now, we're going to develop a model that can detect the following fruits in this Kaggle [dataset](https://www.kaggle.com/datasets/karimabdulnabi/fruit-classification10-class):

- Apple
- Banana
- Avocado
- Cherry
- Kiwi
- Mango
- Orange
- Pineapple
- Strawberries
- Watermelon

**Your tasks:**

In this exercise, you will apply the pre-trained CNN model (`VGG` for this case) to classify images of fruits! 

To begin, you will need to obtain the image, the model's predictions (`Class`), and the associated probabilities by running the following cell.

> If you encounter any error, please rerun the cell

In [ ]:
# Set up data
IMAGE_DIR = {
    "Apple": "/kaggle/input/fruit-classification10-class/MY_data/train/Apple/*.jpeg", 
    "Banana": "/kaggle/input/fruit-classification10-class/MY_data/train/Banana/*.jpeg", 
    "Avocado": "/kaggle/input/fruit-classification10-class/MY_data/train/avocado/*.jpeg", 
    "Cherry": "/kaggle/input/fruit-classification10-class/MY_data/train/cherry/*.jpeg", 
    "Kiwi": "/kaggle/input/fruit-classification10-class/MY_data/train/kiwi/*.jpeg", 
    "Mango": "/kaggle/input/fruit-classification10-class/MY_data/train/mango/*.jpeg", 
    "Orange": "/kaggle/input/fruit-classification10-class/MY_data/train/orange/*.jpeg", 
    "Pineapple": "/kaggle/input/fruit-classification10-class/MY_data/train/pinenapple/*.jpeg", 
    "Strawberries": "/kaggle/input/fruit-classification10-class/MY_data/train/strawberries/*.jpeg", 
    "Watermelon": "/kaggle/input/fruit-classification10-class/MY_data/train/watermelon/*.jpeg", 
}

# Display image and prediction labels with probability
for true_label, image_path in IMAGE_DIR.items():
    show_image_label_prob(image_path, true_label, num_images=2)


**Food for Thought**: 

- How does the model perform on this dataset?
- Do you think it does well in identifying each type of fruit?

_Type your answer here, replacing this text._

### Feature Extractor

**Your tasks:**

In this exercise, you will utilize a pre-trained CNN model (`Densenet` for this case) , to extract features from images. Following this, you will train a machine learning classifier to identify different fruits!

To get started, please run the following cell to prepare the data.

In [ ]:
# Set up data
DATA_DIR = '/kaggle/input/fruit-classification10-class/MY_data/'
SUBDIR = {'train': 'train', 'valid': 'test'}
image_datasets, dataloaders = read_data(DATA_DIR, SUBDIR)
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "valid"]}
class_names = {"train": image_datasets["train"].classes,
               "valid": image_datasets["valid"].classes}

> If you want to take a look at the images in training set, try this code:

In [ ]:
# Plot samples
inputs, classes = next(iter(dataloaders["valid"]))
plt.figure(figsize=(10, 8)); 
plt.axis("off"); 
plt.title("Sample valid Images")
plt.imshow(np.transpose(utils.make_grid(inputs, padding=1, normalize=True),(1, 2, 0)));

Next, utilize the pre-trained model to extract features from the images by executing the following cell.

In [ ]:
# Download model and extract features
model = models.densenet121(weights="DenseNet121_Weights.IMAGENET1K_V1")
model.classifier = nn.Identity()  # remove that last "classification" layer
Z_train, y_train = get_features(
    model, dataloaders["train"], seed=42
)
Z_valid, y_valid = get_features(
    model, dataloaders["valid"], seed=42
)

> If you want to take a look at the extracted features in training set, try this code:

In [ ]:
pd.DataFrame(Z_train).head(10)

To proceed, you will use the extracted features and labels from the image to train a machine learning classifier (`LogisticRegression` for this case). Please run the cell below to begin the training process.

In [ ]:
# Train classification model
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000))
pipe.fit(Z_train, y_train)
print("Training score: ", pipe.score(Z_train, y_train))
pipe.score(Z_valid, y_valid)
print("Validation score: ", pipe.score(Z_valid, y_valid))

> If you want to take a look at the images in validation set and compare the actual labels with the predicted labels, try this code:

In [ ]:
# Show predictions for 25 images in the validation set (5 rows of 5 images)
show_predictions(pipe, Z_valid, y_valid, dataloaders['valid'], class_names, num_images=25, seed=42)

**Food for Thought**: 

- How is the performance of the model?
- How does the out-of-the-box model compare to the feature extractor model? 

_Type your answer here, replacing this text._

### Your Free Time (Optional)

**Your tasks**:

Choose any image dataset that interests you and train a model using it!

Feel free to discuss your ideas and progress with your teammates and the workshop team.

In [ ]:
DATA_DIR = "{_DATA_DIRECTORY_PATH_}"
SUBDIR = "{_SUB_DIRECTORY_PATH_}"
# Example - dataset `cat-breed-mardhik`
# DATA_DIR = '/kaggle/input/cat-breed/cat-breed/'
# SUBDIR = {'train': 'TRAIN', 'valid': 'TEST'}

# Set up data
image_datasets, dataloaders = read_data(DATA_DIR, SUBDIR)
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "valid"]}
class_names = {"train": image_datasets["train"].classes,
               "valid": image_datasets["valid"].classes}

# Download model and extract features
model = models.densenet121(weights="DenseNet121_Weights.IMAGENET1K_V1")
model.classifier = nn.Identity()  # remove that last "classification" layer
Z_train, y_train = get_features(
    model, dataloaders["train"], seed=42
)
Z_valid, y_valid = get_features(
    model, dataloaders["valid"], seed=42
)

# Train classification model
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000))
pipe.fit(Z_train, y_train)
print("Training score: ", pipe.score(Z_train, y_train))
pipe.score(Z_valid, y_valid)
print("Validation score: ", pipe.score(Z_valid, y_valid))

In [ ]:
# Plot samples
show_predictions(pipe, Z_valid, y_valid, dataloaders['valid'], class_names, num_images=25, seed=42)

<!-- END QUESTION -->

<br><br>